In [ ]:
import phd

import os
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import nengo
import numpy as np
from IPython.display import SVG
from scipy import stats
from nengo.utils.matplotlib import rasterplot
from nengo.utils.stdlib import Timer

# Some plotting niceties
phd.plots.setup()

# Ensure TIMIT is extracted
timit_path = "~/phd_data/timit"
timit = phd.timit.TIMIT(timit_path)
try:
    timit.untar(os.path.expanduser("~/Dropbox/LDC93S1.tgz"))
except Exception as e:
    pass

# Some general params
utt = 'deadline'
spkr = 'RRE0'

# Plotting functions
def prod(key, x_label):
    df = phd.analysis.load_results(phd.experiments.ProductionResult, [key])
    cmp_args = {'group_by': key, 'x_label': x_label}

    fig = plt.figure(figsize=(6, 5))
    ax = plt.subplot(2, 1, 1)
    phd.plots.timeseries(df, columns=['accuracy'], x_keys=['Accuracy'], y_label='Accuracy',
                         ax=ax, **cmp_args)
    ax.axhline(phd.analysis.gs_accuracy_baseline, c='k', ls=':')
    ax.set_xlabel("")
    ax.set_xticks(())
    l = ax.legend()
    l.remove()

    ax = plt.subplot(2, 1, 2)
    phd.plots.timeseries(df, columns=['cooccur'], x_keys=['Co-occurrence'], y_label='Co-occurrence',
                         ax=ax, **cmp_args)
    ax.axhline(df["co_chance"].mean(), c='k', ls=":")
    l = ax.legend()
    l.remove()
    ax.set_xlabel(x_label)

    fig.tight_layout()
    fig.savefig(os.path.join(phd.plots.root, "figures", "presentation", "prod-%s.svg" % key))

def recog(key, x_label, n_syllables=[3., 3.]):
    df = phd.analysis.load_results(phd.experiments.RecognitionResult, [key])
    cmp_args = {'group_by': key, 'x_label': x_label}

    fig = plt.figure(figsize=(6, 5))
    ax = plt.subplot(2, 1, 1)
    phd.plots.timeseries(df, columns=['acc'], x_keys=['Accuracy'], y_label='Accuracy',
                         ax=ax, **cmp_args)
    ax.set_xlabel("")
    ax.set_xticks(())
    ax.set_ylim(0, 1)
    l = ax.legend()
    l.remove()

    ax = plt.subplot(2, 1, 2)
    phd.plots.timeseries(df, columns=['memory_acc'], x_keys=['Memory accuracy'],
                         y_label='Memory accuracy', ax=ax, **cmp_args)
    l, r = ax.get_xlim()
    s = np.asarray(n_syllables)
    x = np.linspace(l, r, s.size)
    ax.plot(x, 1./s, c='k', ls=':')
    l = ax.legend()
    l.remove()
    ax.set_xlabel(x_label)

    fig.tight_layout()
    fig.savefig(os.path.join(phd.plots.root, "figures", "presentation", "recog-%s.svg" % key))

# NCC

In [ ]:
timit = phd.timit.TIMIT(timit_path)
timit.filefilt.spkr_id = spkr
samples = timit.word_samples([utt])
model = phd.sermo.AuditoryFeatures()
model.audio = samples[utt][0]

In [ ]:
# Get MFCCs
model.mfcc.dt = 0.01
x = model.mfcc()
n_frames = x.shape[0]
print x.shape

# Plot
f, ax = phd.plots.plot_traj(x, zscore=True)
f.set_size_inches(6, 2.5)
ax.set_ylabel("MFCC")
ax.set_xlabel("Frame")
f.tight_layout()
phd.plots.savefig(f, 'presentation', 'ncc-mfcc')

In [ ]:
# NCCs shortened to 31 frames
net = model.build(nengo.Network(seed=0))
with net:
    ihc_p = nengo.Probe(net.periphery.ihc, synapse=None)
    an_in_p = nengo.Probe(net.periphery.an.input, synapse=None)
    an_p = nengo.Probe(net.periphery.an.add_neuron_output(), synapse=None)
    c_p = nengo.Probe(net.output, synapse=0.01)
sim = nengo.Simulator(net, dt=0.001)
sim.run(model.t_audio)
ncc = phd.experiments.shorten(sim.data[c_p], n_frames=n_frames)

# Plot
t, t_ix = sim.trange(model.t_audio / n_frames), np.arange(0, n_frames, 10)
t_ix[1:] -= 1
f, ax = phd.plots.plot_traj(ncc, zscore=False)
f.set_size_inches(6, 2.5)
ax.set_ylabel("NCC")
ax.set_xticks(t_ix)
ax.set_xticklabels(np.round(t[t_ix], 3))
ax.set_xlabel("Time (s)")
f.tight_layout()
phd.plots.savefig(f, 'presentation', 'ncc-ncc')

In [ ]:
w = 6 * phd.figures.in2px
h = 2.42 * phd.figures.in2px

fig = phd.figures.svgfig(w, h * 2)
fig.append(phd.figures.el(None, phd.figures.svgpath('ncc-mfcc', subdir='presentation'), 0, 0))
fig.append(phd.figures.el(None, phd.figures.svgpath('ncc-ncc', subdir='presentation'), 0, h))
phd.figures.savefig(fig, 'mfcc-ncc', subdir='presentation')
SVG(os.path.join(phd.figures.root, 'figures', 'presentation', 'mfcc-ncc.svg'))

In [ ]:
phd.plots.setup(figsize=(5, 3.5))
pargs = {'columns': [], 'vary': 'phones', 'filter_by': [], 'hue_order': ["vowels", "consonants", "all"]}

def fix_label(l):
    for t, label in zip(l.get_texts(), ["Vowels", "Consonants", "All phones"]):
        t.set_text(label)

_, b = phd.plots.ncc_accuracy(relative=False, **pargs)
ax = b.get_axes()[0]
ax.set_ylabel("Classification correctness")
l = ax.legend(title="", bbox_to_anchor=(0.55, 0.9), bbox_transform=b.transFigure)
fix_label(l)
b.savefig(os.path.join(phd.plots.root, "figures", "presentation", "ncc-phones-acc-b.svg"))

_, b = phd.plots.ncc_accuracy(relative=True, **pargs)
ax = b.get_axes()[0]
ax.set_ylabel("Relative classification correctness")
l = ax.legend(title="")
fix_label(l)
b.savefig(os.path.join(phd.plots.root, "figures", "presentation", "ncc-phones-racc-b.svg"))

t = phd.plots.ncc_time(**pargs)
ax = t.get_axes()[0]
l = ax.legend(loc='upper left', title="")
fix_label(l)
ax.set_xlim(right=1.5)
t.savefig(os.path.join(phd.plots.root, "figures", "presentation", "ncc-phones-time.svg"))

# Syllable production

In [ ]:
with nengo.Network() as model:
    ramp = nengo.Node(lambda t: np.clip(t - 0.1, 0, 1))
    ens_1d = nengo.Ensemble(200, dimensions=1)
    nengo.Connection(ramp, ens_1d)
    p = nengo.Probe(ens_1d, synapse=0.01)
sim = nengo.Simulator(model)
sim.run(1.2)

plt.figure(figsize=(6, 3))
plt.plot(sim.trange(), sim.data[p])
plt.xlabel("Time (s)")
sns.despine()
plt.tight_layout()
plt.savefig(os.path.join(phd.plots.root, 'figures', 'presentation', 'ramp.svg'))

In [ ]:
path = phd.ges_path('ges-de-cvc', 'ban.ges')
score = phd.vtl.parse_ges(path) 
traj = score.trajectory(dt=0.001)

plt.figure(figsize=(6, 3))
plt.pcolormesh(traj.T, linewidth=0, rasterized=True)
plt.xticks(np.linspace(0, 1200, 6), np.linspace(0, 1, 6))
plt.ylabel("Gesture index")
plt.xlabel("Relative time")
sns.despine()
plt.tight_layout()
plt.savefig(os.path.join(phd.plots.root, 'figures', 'presentation', 'gs-traj.svg'))

In [ ]:
prod('freq', 'Frequency')

In [ ]:
prod('sequence_len', 'Sequence length')

In [ ]:
prod('n_syllables', 'Syllabary size')

# Syllable classification

In [ ]:
recog('sequence_len', 'Sequence length')

In [ ]:
recog('freq', 'Syllable frequency')